In [61]:
# -*- coding: utf-8 -*-
from __future__ import unicode_literals
import codecs
import os
import re
OUTPUT_PATH='data'

'''
TRAIN_DATASET='original_data/msr_training.utf8'
TEST_GOLD_DATASET='original_data/msr_test_gold.utf8'
TEST_DATASET='original_data/msr_test.utf8'
'''
TRAIN_DATASET='original_data/bala_training.utf8'
TEST_GOLD_DATASET='original_data/bala_test_gold.utf8'
TEST_DATASET='original_data/bala_test.utf8'

CHINESE_IDIOMS='original_data/idioms'


rNUM = '(-|\+)?\d+((\.|·)\d+)?%?'
rENG = '[A-Za-z_.]+'


In [65]:
def strQ2B(ustring):
    """全角转半角"""
    rstring = ""
    for uchar in ustring:
        inside_code = ord(uchar)#返回对应的 ASCII 数值，或者 Unicode 数值
        if inside_code == 12288:  # 全角空格直接转换
            inside_code = 32
        elif (inside_code >= 65281 and inside_code <= 65374):  # 全角字符（除空格）根据关系转化
            inside_code -= 65248
        rstring += unichr(inside_code)
    return rstring

In [66]:
def preprocess(input,output):
    output_filename = os.path.join(OUTPUT_PATH,output)

    #取出成语词典中的成语
    idioms=dict()
    with codecs.open(CHINESE_IDIOMS,'r','utf-8') as f:    
        for line in f:  
            idioms[line.strip()]=1    #返回移除字符串头尾指定的字符(空格)生成的新字符串                              标记为1
    
    count_idioms = 0
    sents=[]
    #pku_training.utf8的文本处理：
    with codecs.open(input,'r','utf-8') as fin:
        with codecs.open(output_filename,'w','utf-8') as fout:
            for line in fin:   #取pku_training 的每一行
                sent=strQ2B(line).split( ) #此行 词 的集合
                new_sent=[]
                for word in sent:
                    #word=re.sub(rNUM,'0',word)  #正则 没懂 ？？？？？？？？？？？？？？？？
                    #word=re.sub(rENG,'X',word)
                    if idioms.get(word) is not None:
                        count_idioms+=1
                        word=u'I'
                    new_sent.append(word)
                sents.append(new_sent)
            for sent in sents:
                fout.write('  '.join(sent))
                fout.write('\n')
    #print 'idioms count:%d' % count_idioms

In [67]:
preprocess(TRAIN_DATASET,'dataset_train_all') 

NameError: name 'unichr' is not defined